### Library Import

In [1]:
import pandas as pd
import csv
import numpy as np
import os
import pymysql

from dataloader.dataloader import data_loader
from utils.data_split import data_split
from utils.load_params import load_params
from models.train_model import train_model
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_squared_log_error, mean_absolute_percentage_error, explained_variance_score
from datetime import datetime

### DB connect

In [2]:
hostname = "10.28.224.84"
port = 30234
user = "dogeol" # dogeol/
password = "1234"
db = "testdb"

mysql = pymysql.connect(host = hostname, port = port, user = user, password = password, db = db)
cursor = mysql.cursor(pymysql.cursors.DictCursor)

### Setting

In [3]:
dataset_name = "pure" # pure
model_name = "LGBM" # LGBM/
split_type = "time" # random/time
model_type = "regressor" # classifier/regressor

In [4]:
now = datetime.now()
date_code = "".join([str(now.month), str(now.day), str(now.hour), str(now.minute)])
save_name = "_".join([model_name, dataset_name, split_type, date_code, ".csv"])

### Dataset Load

In [5]:
train_df, test_df, submission_df , drop_columns, target_column = data_loader(dataset_name)

In [6]:
x_train, x_valid, y_train, y_valid = data_split(split_type, train_df, target_column)
X_test = test_df.copy()

### Model Training

In [7]:
params = load_params(model_name, model_type)
model = train_model(model_name, model_type, params, x_train, y_train)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.008439 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 972
[LightGBM] [Info] Number of data points in the train set: 1594362, number of used features: 8
[LightGBM] [Info] Start training from score 37814.733261


### Training Result

In [8]:
y_valid_pred = model.predict(x_valid)

y_valid_mae = mean_absolute_error(y_valid, y_valid_pred)
y_valid_mse = mean_squared_error(y_valid, y_valid_pred)
y_valid_rmse = np.sqrt(mean_squared_error(y_valid, y_valid_pred))
y_valid_r2 = r2_score(y_valid, y_valid_pred)
y_valid_msle = mean_squared_log_error(y_valid, y_valid_pred)
y_valid_mape = mean_absolute_percentage_error(y_valid, y_valid_pred)
y_valid_evs = explained_variance_score(y_valid, y_valid_pred)

In [9]:
print(f"MAE: {y_valid_mae:.2f}") # 절대 오차의 평균, 값이 작을수록 좋음
print(f"MSE: {y_valid_mse:.2f}") # 제곱 오차의 평균, 값이 작을수록 좋음
print(f"RMSE: {y_valid_rmse:.2f}") # MSE의 제곱근, 값이 작을수록 좋음
print(f"R2: {y_valid_r2:.2f}") # 모델의 설명력, 값이 1에 가까울수록 예측이 정확함
print(f"MSLE: {y_valid_msle:.2f}") # 예측값과 실제값의 로그 차이를 기반으로 한 평균 제곱 오차, 값이 작을수록 좋음
print(f"MAPE: {y_valid_mape:.2f}") # 절대 오차를 실제 값에 대한 백분율로 나타낸 값, 값이 작을수록 좋음
print(f"EVS: {y_valid_evs:.2f}") # 예측된 값과 실제 값 사이의 분산을 측정, 값이 1에 가까울수록 예측이 정확함

MAE: 6549.25
MSE: 109240168.27
RMSE: 10451.80
R2: 0.87
MSLE: 0.06
MAPE: 0.18
EVS: 0.88


### Save Result

In [10]:
x_total = pd.concat([x_train, x_valid], axis=0)
y_total = pd.concat([y_train, y_valid], axis=0)
model = train_model(model_name, model_type, params, x_total, y_total)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.010513 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 980
[LightGBM] [Info] Number of data points in the train set: 1801228, number of used features: 8
[LightGBM] [Info] Start training from score 38162.229423


In [12]:
date = now.strftime('%Y-%m-%d %H:%M:%S')
try:
    cursor.execute("""
        INSERT INTO recsys_02.result 
        (date, user, save_name, MAE, MSE, RMSE, R2, MSLE, MAPE, EVS, leaderboard, params) 
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """, 
    (
        date, 
        user, 
        save_name, 
        round(y_valid_mae, 2), 
        round(y_valid_mse, 2), 
        round(y_valid_rmse, 2), 
        round(y_valid_r2, 2), 
        round(y_valid_msle, 2), 
        round(y_valid_mape, 2), 
        round(y_valid_evs, 2), 
        0, 
        str(params)
    ))

    mysql.commit()
    print("Your data has been saved successfully.")

except Exception as e:
    print(f"error : {e}")

finally:
    cursor.close()
    mysql.close()


데이터가 성공적으로 저장되었습니다.


In [13]:
test_pred = model.predict(X_test)
submission_df['deposit'] = test_pred
submission_df.to_csv(save_name, index=False, encoding='utf-8-sig')